# Setup

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Load data

In [ ]:
df = pd.read_csv('../data/final.csv', index_col=['state'])

# Plot

In [ ]:
df.sort_values('pct_of_pop', ascending=False).plot();

In [ ]:
(
    df
    .sort_values('pct_of_pop', ascending=False)
    .assign(
        difference=lambda df: df.pct_of_electors.div(df.pct_of_pop),
        cum_sum=lambda df: df.pct_of_pop.cumsum()
    )
    .plot('cum_sum', 'difference')
)
plt.xticks(np.arange(101, step=5))
plt.xlabel('Percent of U.S. Population (Cumulative)')
plt.ylabel('Representativeness');

In [ ]:
(
    df
    .sort_values('pct_of_pop')
    .assign(
        difference=lambda df: df.pct_of_electors.div(df.pct_of_pop).sub(1),
    )
    .difference
    .plot.barh(figsize=(8, 14))
)

plt.xticks(np.arange(-0.5, 3, 0.5), ['{}%'.format(x) for x in np.arange(50, 400, 50)])
plt.gca().yaxis.grid(False)

plt.axvline(0, c='k')

plt.xlabel('Representativeness (% of Electors - % of Populaton)')
plt.ylabel('States (Ordered from Highest Population to Lowest)')
plt.title('Over- vs. Under-Representation in the Electoral College');

In [ ]:
plot_df = (
    df
    .sort_values('pct_of_pop')
    .assign(difference=lambda df: df.pct_of_electors.div(df.pct_of_pop).sub(1))
)

def colors(series):
    c = pd.Series(index=series.index)
    
    c.loc[series.lt(0)] = '#ef8a62'
    c.loc[series.eq(0)] = '#f7f7f7'
    c.loc[series.gt(0)] = '#67a9cf'
    
    return c.tolist()

(
    plot_df
    .difference
    .plot.barh(figsize=(8, 14), color=plot_df.difference.pipe(colors))
)

plt.xticks(np.arange(-0.5, 3, 0.5), ['{}%'.format(x) for x in np.arange(50, 400, 50)])
plt.gca().yaxis.grid(False)

plt.xlabel(r'$Representativeness \/ \left(\frac{\% \/ of \/ Electors}{\% \/ of \/ Populaton}\right)$')
plt.ylabel('States (Ordered from Highest Population to Lowest)')
plt.title('Over- vs. Under-Representation in the Electoral College');

In [ ]:
def proportional_difference(df):
    difference = df.pct_of_electors.div(df.pct_of_pop)
    
    lt = difference.lt(1)
    eq = difference.eq(1)
    gt = difference.gt(1)
    
    difference.loc[lt] = difference.loc[lt].rdiv(1).rsub(1)
    difference.loc[eq] = 0
    difference.loc[gt] = difference.loc[gt].sub(1)
    
    return difference

plot_df = (
    df
    .sort_values('pct_of_pop')
    .assign(
        difference=lambda df: df.pct_of_electors.div(df.pct_of_pop).sub(1),
        prop_diff=proportional_difference,
    )
)

def colors(series):
    c = pd.Series(index=series.index)
    
    c.loc[series.lt(0)] = '#ef8a62'
    c.loc[series.eq(0)] = '#f7f7f7'
    c.loc[series.gt(0)] = '#67a9cf'
    
    return c.tolist()

(
    plot_df
    .prop_diff
    .plot.barh(figsize=(8, 14), color=plot_df.prop_diff.pipe(colors))
)

plt.xticks(np.arange(-0.5, 3, 0.5), ['{}%'.format(x) for x in np.arange(50, 400, 50)])
plt.gca().yaxis.grid(False)

plt.xlabel('Representativeness (adjusted)')
plt.ylabel('States (Ordered from Highest Population to Lowest)')
plt.title('Over- vs. Under-Representation in the Electoral College');

In [ ]:
plot_df = (
    df
    .sort_values('pct_of_pop')
    .assign(
        difference=lambda df: df.pct_of_electors.div(df.pct_of_pop).sub(1),
        president=lambda df: df.president.fillna('Inconclusive')
    )
)

def colors(series):
    c = pd.Series({'Rep': '#ef8a62', 'Dem': '#67a9cf', 'Inconclusive': '#f7f7f7'})
    
    return series.map(c).tolist()

def build_legend():
    dem = mpatches.Patch(color='#67a9cf', label='Democrat')
    rep = mpatches.Patch(color='#ef8a62', label='Republican')
    inc = mpatches.Patch(color='#f7f7f7', label='Inconclusive')
    
    return plt.legend(handles=[dem, inc, rep])

(
    plot_df
    .difference
    .plot.barh(figsize=(8, 14), color=plot_df.president.pipe(colors))
)

plt.xticks(np.arange(-0.5, 3, 0.5), ['{}%'.format(x) for x in np.arange(50, 400, 50)])
plt.gca().yaxis.grid(False)

plt.xlabel(r'$Representativeness \/ \left(\frac{\% \/ of \/ Electors}{\% \/ of \/ Populaton}\right)$')
plt.ylabel('States (Ordered from Highest Population to Lowest)')
plt.title('Over- vs. Under-Representation in the Electoral College')

build_legend();

In [ ]:
plot_series = (
    df
    .sort_values('pct_of_pop')
    .assign(
        difference=lambda df: df.pct_of_electors.div(df.pct_of_pop),
        president=lambda df: df.president.fillna('Inconclusive')
    )
    .groupby('president')
    .difference
    .mean()
)

def colors(series):
    c = pd.Series({'Rep': '#ef8a62', 'Dem': '#67a9cf', 'Inconclusive': '#f7f7f7'})
    
    return c.reindex(series.index).tolist()

(
    plot_series
    .plot.bar(rot=0, ylim=(0, 1.5), color=plot_series.pipe(colors))
)

plt.yticks(np.arange(0, 1.6, 0.2), ['{}%'.format(x) for x in np.arange(0, 160, 20)])

plt.xlabel('Party')
plt.ylabel('Mean Representativeness')
plt.title('Over- vs. Under-Representation in the Electoral College');

In [ ]:
plot_series = (
    df
    .sort_values('pct_of_pop')
    .assign(
        difference=lambda df: df.pct_of_electors.div(df.pct_of_pop),
        president=lambda df: df.president.fillna('Inconclusive')
    )
    .groupby('president')
    .size()
)

def colors(series):
    c = pd.Series({'Rep': '#ef8a62', 'Dem': '#67a9cf', 'Inconclusive': '#f7f7f7'})
    
    return c.reindex(series.index).tolist()

(
    plot_series
    .plot.bar(rot=0, ylim=(0, 50), color=plot_series.pipe(colors))
)

plt.xlabel('Party')
plt.ylabel('Number of states')
plt.title('Over- vs. Under-Representation in the Electoral College');